In [1]:
import os
import sys
import time
import pickle 
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd

# Kernel ridge regression

In [ ]:
%matplotlib inline

# ========================= Setting up the parameters

atoms = ['Na','I','Cs']
kernel = 'poly'
degree = 1

# ========================= End of setting up the parameters

all_errors_ = []
for atom in atoms:
    guesses = ['atomic','xtb']
    props = ['overlap','density','ham']
    all_inputs = []
    labels = []
    for guess in guesses:
        for prop in props:
            inputs = np.load(f'../1_generate_data/{atom}/{atom}_{guess}_guess_{prop}.npy')
            all_inputs.append(inputs)
            labels.append(f'{guess} {prop}')
    outputs = np.load(f'../1_generate_data/{atom}_ref_efg.npy')
    ntrains = []
    for i in [1,2.5,5.0,10.0,25.0,50.0]: # in percentage
        ntrains.append(int(i*inputs.shape[0]/100))
    shuffled_indices = np.load(f'{atom}_shuffled_indices.npy')
    all_outputs = []
    all_errors = []
    all_models = []
    all_input_scalers = []
    all_output_scalers = []
    for i1 in range(len(all_inputs)):
        print(labels[i1])
        errors = []
        inputs = all_inputs[i1]
        tmp = []
        tmp_model = []
        tmp_in_scale = []
        tmp_out_scale = []
        for ntrain in ntrains:
            # train_indices = np.linspace(0, inputs.shape[0]-1, ntrain, dtype=int)
            train_indices = list(shuffled_indices[0:ntrain])
            if max(shuffled_indices) not in train_indices:
                train_indices.append(max(shuffled_indices))
            if min(shuffled_indices) not in train_indices:
                train_indices.append(min(shuffled_indices))
            test_indices = list(set(range(0,inputs.shape[0]))-set(train_indices))
            # Initializing the scaler
            input_scaler = StandardScaler()     # MinMaxScaler()
            output_scaler = StandardScaler()    # MinMaxScaler()
            input_scaler.fit(inputs[train_indices])
            output_scaler.fit(outputs[train_indices])

            # Scaling the training set
            inputs_train_scaled = input_scaler.transform(inputs[train_indices])
            outputs_train_scaled = output_scaler.transform(outputs[train_indices])

            # Scaling the testing set
            inputs_test_scaled = input_scaler.transform(inputs[test_indices])
            outputs_test_scaled = output_scaler.transform(outputs[test_indices])
            model = KernelRidge(kernel='poly', degree=degree, alpha=0.1, gamma=0.1)
            model.fit(inputs_train_scaled, outputs_train_scaled)
            predictions_train = model.predict(inputs_train_scaled)  
            predictions_train_scaled = output_scaler.inverse_transform(predictions_train)
            # Mean square error
            mse = np.sqrt(mean_squared_error(outputs[train_indices], predictions_train_scaled))
            # Mean absolute error
            acc = mean_absolute_error(outputs[train_indices], predictions_train_scaled)
            #print(F'Model ---> Training    MSE:', mse, '  MAE:', acc)
            predictions_test = model.predict(inputs_test_scaled)
            predictions_test_scaled = output_scaler.inverse_transform(predictions_test)
            # Mean square error
            mse = np.sqrt( mean_squared_error(outputs[test_indices], predictions_test_scaled) )
            # Mean absolute error
            acc = mean_absolute_error(outputs[test_indices], predictions_test_scaled)
            #print(F'Model ---> Testing     MSE:', mse, '  MAE:', acc)
            inputs_scaled = input_scaler.transform(inputs)
            predictions = model.predict(inputs_scaled)
            predictions_scaled = output_scaler.inverse_transform(predictions)
            # Mean square error
            mse = np.sqrt( mean_squared_error(outputs, predictions_scaled) )
            # Mean absolute error
            acc = mean_absolute_error(outputs, predictions_scaled)
            #print(F'Model ---> All     MSE:', mse, '  MAE:', acc)
            #print('#############')
            errors.append([acc,mse])
            tmp.append(predictions_scaled)
            tmp_model.append(model)
            tmp_in_scale.append(input_scaler)
            tmp_out_scale.append(output_scaler)
            # ================================ Plotting
            plt.figure(figsize=(3.21*6,2.41*3))
            x = input_scaler.transform(inputs)
            y = model.predict(x)
            y = output_scaler.inverse_transform(y)
            labels_ = ['Vxx','Vxy','Vxz','Vyx','Vyy','Vyz','Vzx','Vzy','Vzz']
            df = pd.DataFrame(y, columns=['Vxx','Vxy','Vxz','Vyx','Vyy','Vyz','Vzx','Vzy','Vzz'])
            df.to_csv(f'ml_efg_{atom}_guess_{guess}_prop_{prop}_kernel_{kernel}_degree_{degree}_ntrain_{ntrain}.csv', 
                      float_format='%.12f', index=True)
            prop = labels[i1].split()[1]
            guess = labels[i1].split()[0]
            plt.rcParams.update({'lines.linewidth': 1.0})
            for i in range(9):
                plt.subplot(3,3,i+1)
                plt.plot(outputs[:,i], label='Ref', color='blue')
                plt.plot(y[:,i], label='Prediction', color='red', ls='--')
                plt.title(labels_[i])
            if degree==1:
                plt.suptitle(f'{atom}, {guess} guess, {prop}, Ntrain: {ntrain}, linear kernel')
            elif degree>1:
                plt.suptitle(f'{atom}, {guess} guess, {prop}, Ntrain: {ntrain}, $x^{degree}$ kernel')
            plt.subplot(3,3,3)
            plt.legend()
            plt.tight_layout()
            plt.savefig(f'A1_{atom}_{guess}_guess_{prop}_ntrain_{ntrain}_kernel_{kernel}_degree_{degree}_more_elements.jpg',
                        dpi=300)
            #plt.close()
            # ================================ Plotting
        all_outputs.append(tmp)
        #errors = np.array(errors)
        all_errors.append(errors)
        all_input_scalers.append(tmp_in_scale)
        all_output_scalers.append(tmp_out_scale)
        all_models.append(tmp_model)
    all_errors = np.array(all_errors)
    all_outputs = np.array(all_outputs)
    print(all_errors.shape, all_outputs.shape)
    colors = ['blue','red','green','purple', 'orange', 'olive']
    all_errors_.append(all_errors)
all_errors_ = np.array(all_errors_)
np.save(f'mae_degree_{degree}.npy', all_errors_)